# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:

* Compute important summary statistics on the complete data set (using `graphlab SArray` and `SFrame` functions or any other modules)
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complete is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

When using `pandas`, there is no need to load the housing data as the train and the test data are provided as part of the course. This was done to avoid the random seed creating issue in split as the quiz answers are based on that.

## Modules

In [1]:
# import graphlab

import pandas as pd
import numpy as np

## Get Data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
# sales = graphlab.SFrame('kc_house_data.gl/')
sales = pd.read_csv("./data/kc_house_train_data.csv")
train_data = pd.read_csv("./data/kc_house_train_data.csv")
test_data = pd.read_csv("./data/kc_house_test_data.csv")

## Summary Statistics

In [3]:
sales.shape

(17384, 21)

In [4]:
sales.columns

Index([u'id', u'date', u'price', u'bedrooms', u'bathrooms', u'sqft_living',
       u'sqft_lot', u'floors', u'waterfront', u'view', u'condition', u'grade',
       u'sqft_above', u'sqft_basement', u'yr_built', u'yr_renovated',
       u'zipcode', u'lat', u'long', u'sqft_living15', u'sqft_lot15'],
      dtype='object')

In [5]:
sales.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [6]:
sales.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,1.738400e+04,1.738400e+04,17384.000000,17384.000000,17384.000000,1.738400e+04,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000,17384.000000
mean,4.574349e+09,5.393666e+05,3.369363,2.115048,2080.029510,1.509191e+04,1.494248,0.007651,0.236079,3.410780,7.655028,1787.844512,292.184998,1971.152727,83.107973,98077.936896,47.559313,-122.213281,1985.994995,12776.380867
std,2.872356e+09,3.696912e+05,0.906468,0.771783,921.630888,4.145927e+04,0.539443,0.087136,0.768008,0.649792,1.169818,827.107595,444.404136,29.328722,398.692283,53.525617,0.138703,0.140906,686.512835,27175.730523
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.159300,-122.519000,399.000000,651.000000
25%,2.124087e+09,3.200000e+05,3.000000,1.750000,1420.000000,5.049500e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1200.000000,0.000000,1952.000000,0.000000,98033.000000,47.468650,-122.328000,1490.000000,5100.000000
50%,3.892800e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.616000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571400,-122.229000,1840.000000,7620.000000
75%,7.304301e+09,6.400000e+05,4.000000,2.500000,2550.000000,1.066525e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98117.000000,47.677625,-122.125000,2360.000000,10065.250000
max,9.900000e+09,7.700000e+06,10.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


#### Split data into training and testing - needed only for graphlab create

We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you). 

    train_data,test_data = sales.random_split(.8,seed=0)

In [7]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales['price'] # extract the price column

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = prices.size # .size() when using SArrays
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print "average price via method 1: " + str(avg_price_1)
print "average price via method 2: " + str(avg_price_2)

average price via method 1: 539366.627934
average price via method 2: 539366.627934


As we see we get the same answer both ways

In [8]:
# Let's compute the sum of squares of price. We can multiply two SArrays of the same length elementwise also with *
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum() # price_squared is an SArray of the squares and we want to add them up.
print "the sum of price squared with scientific notation is: " + str(sum_prices_squared)
print "the sum of price squared without scientific notation is: " + str('{0:f}'.format(sum_prices_squared))

the sum of price squared with scientific notation is: 7.43305185234e+15
the sum of price squared without scientific notation is: 7433051852335772.000000


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

## Build a generic simple linear regression function 

We can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [9]:
def simple_linear_regression(X, Y):
    
    """
    X is the input feature
    Y is the output
    small caps in the formula represent means.
    """
    
    # compute the mean of input_feature and output
    x = X.mean()
    y = Y.mean()
    
    # compute the product of the output and the input_feature and then take its mean
    XY =  X * Y
    xy = XY.mean()
    
    # compute the squared value of the input_feature and its mean
    sq_X = X ** 2
    sq_x = sq_X.mean()
    
    numerator = xy - (x * y)
    denominator = sq_x - (x * x)
    
    # use the formula for the slope
    slope = numerator / denominator

    # use the formula for the intercept
    intercept = y - slope * x
    
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [10]:
# test_feature = graphlab.SArray(range(5))
# test_output = graphlab.SArray(1 + 1*test_feature)
test_feature = pd.Series(range(5))
test_output = (1 + test_feature)

(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print "Intercept: " + str(test_intercept)
print "Slope: " + str(test_slope)

Intercept: 1.0
Slope: 1.0


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [11]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print "Intercept: " + str(sqft_intercept)
print "Slope: " + str(sqft_slope)

Intercept: -47116.0790729
Slope: 281.95883963


### Predicting Values

Now that we have the model parameters: `intercept` & `slope` we can make predictions. Using SArrays it's easy to multiply an SArray by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [12]:
def get_regression_predictions(X, w0, w1):
    """
    X is the input feature
    w0 is the intercept consistent with the notation in the lectures
    w1 is the intercept consistent with the notation in the lectures
    """
    # calculate the predicted values:
    
    y_pred = w0 + (w1*X)
    
    return y_pred

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

## Quiz Question 1

Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [13]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print "The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price)

The estimated price for a house with 2650 squarefeet is $700074.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [14]:
def get_residual_sum_of_squares(X, Y, w0, w1):
    """
    X is the input feature
    Y is the actual value
    w0 is the intercept consistent with the notation in the lectures
    w1 is the intercept consistent with the notation in the lectures
    """
    # First get the predictions
    y_pred = w0 + (w1*X)

    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    error = y_pred - Y

    # square the residuals and add them up
    RSS = (error ** 2).sum()

    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [15]:
print get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope) # should be 0.0

0.0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

## Quiz Question 2: 

According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [16]:
rss_prices_on_sqft = get_residual_sum_of_squares(
    train_data['sqft_living'],
    train_data['price'],
    sqft_intercept,
    sqft_slope)

print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft)

The RSS of predicting Prices based on Square Feet is : 1.20191835418e+15


### Predict the squarefeet given price

What if we want to predict the squarefoot given the price? 

Since we have an equation y = a + b*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [17]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    
    estimated_feature = (output - intercept) / slope
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

## Quiz Question 3

According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [18]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print "The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet)

The estimated squarefeet for a house worth $800000.00 is 3004


## New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [19]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'

bed_intercept, bed_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])
print "Intercept: " + str(bed_intercept)
print "Slope: " + str(bed_slope)

Intercept: 109473.177623
Slope: 127588.952934


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

## Quiz Question 4

Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [20]:
# Compute RSS when using bedrooms on TEST data:


bed_intercept, bed_slope = simple_linear_regression(test_data['bedrooms'], test_data['price'])
print get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], bed_intercept, bed_slope)


4.9059714283e+14


In [21]:
# Compute RSS when using squarefeet on TEST data:

sqft_intercept, sqft_slope = simple_linear_regression(test_data['sqft_living'], test_data['price'])
print get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], sqft_intercept, sqft_slope)


2.751685739e+14
